### Load an RDF graph

In this example, we will use the `ipyradiant` `FileManager`.

In [ ]:
from ipyradiant import FileManager, PathLoader

lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
lw

In [ ]:
from ipyradiant.query.api import SPARQLQueryFramer, build_values
from ipyradiant.query.meta import MetaTriplesWithValues

from rdflib import Namespace
from rdflib.namespace import RDF, RDFS

In [ ]:
sw_hum = Namespace("https://swapi.co/resource/human/")
VOC = Namespace("https://swapi.co/vocabulary/")

class CustomQuery(SPARQLQueryFramer, metaclass=MetaTriplesWithValues):
    values={
        "s": [sw_hum+'1', sw_hum+'4'],
        "p": [RDF.type]*2
    }

In [ ]:
CustomQuery.run_query(lw.graph)